In [1]:
import random
from dgraph import readGraph
from dgraph import standardize
from dgraph import Page_Rank as pr
from dgraph import fill_incoming as fi
from dgraph import Independent_Cascade as ic
from dgraph import GenWSGridGraph as genWS
from dgraph import WS2DGraph as WS2D
from lesson4 import topk
from lesson5 import eigenvector_centrality as ec
from lesson4 import betweenness
from lesson1 import averageClustering as ac
from dgraph import toUndirect as und

In [2]:
def independent_cascade(graph, seed, centrality):
    adopters, haters, steps = ic(g, seed)
    print 'Independent Cascade Model: TOP %d %s' % (len(seed), centrality)
    print '\tFinal Adopters:\t', len(adopters)
    print '\tFinal Haters:\t', len(haters)
    print '\t# Iterations:\t', steps
    return len(adopters)

In [3]:
NODES = 7115
edges = random.randint(75000, 125000)
radius = 2
weak_ties = [i*5 for i in xrange(0, 3)]
seed = 100

In [4]:
g = standardize(WS2D(NODES, edges, radius, weak_ties))
print ac(und(g))

0.224627301009


In [5]:
fi(g)

In [6]:
print 'Page Rank execution...'
pagerank, iterations, err = pr(g, alpha=1.0e-5, eps=1.0e-3)
print iterations, ' iterations. Error:', err
top_pr = [a for a,b in topk(pagerank, seed)]

Page Rank execution...
31  iterations. Error: 0.000761425430267


In [7]:
print 'Eigenvector Centrality...',
cscores, diffsum = ec(g)
top_eigenc = [a for a, b in topk(cscores, seed)]
print 'Done'

Eigenvector Centrality... Done


In [8]:
print 'Betweennes centrality...',
bet = betweenness(g)
top_bet = [a for a, b in topk(bet, seed)]
print 'Done'

Betweennes centrality... Done


In [9]:
seed = 100
max_pr_ad = independent_cascade(g, top_pr[:seed], 'Page Rank')
max_eigenc_ad = independent_cascade(g, top_eigenc[:seed], 'Eigenvector')
max_bet_ad = independent_cascade(g, top_bet[:seed], 'Betweenness')

Independent Cascade Model: TOP 100 Page Rank
	Final Adopters:	247
	Final Haters:	6868
	# Iterations:	12
Independent Cascade Model: TOP 100 Eigenvector
	Final Adopters:	203
	Final Haters:	6912
	# Iterations:	8
Independent Cascade Model: TOP 100 Betweenness
	Final Adopters:	285
	Final Haters:	6830
	# Iterations:	11


In [10]:
while seed > 0:
    seed -= 5
    adopters_pr = independent_cascade(g, top_pr[:seed], 'Page Rank')
    if max_pr_ad <= adopters_pr:
        max_pr_ad = adopters_pr
    else:
        break
        

Independent Cascade Model: TOP 95 Page Rank
	Final Adopters:	235
	Final Haters:	6880
	# Iterations:	8


In [11]:
seed = 100
while seed > 0:
    seed -= 5
    adopters_eigenc = independent_cascade(g, top_eigenc[:seed], 'Eigenvector')
    if max_eigenc_ad <= adopters_eigenc:
        max_eigenc_ad = adopters_eigenc
    else:
        break
         

Independent Cascade Model: TOP 95 Eigenvector
	Final Adopters:	181
	Final Haters:	6934
	# Iterations:	4


In [12]:
   
seed = 100
while seed > 0:
    seed -= 5
    adopters_bet = independent_cascade(g, top_bet[:seed], 'Betweenness')
    if max_bet_ad <= adopters_bet:
        max_bet_ad = adopters_bet
    else:       
        break
    

Independent Cascade Model: TOP 95 Betweenness
	Final Adopters:	197
	Final Haters:	6918
	# Iterations:	5
